In [2]:
import os
os.chdir('..')

In [18]:
import pandas as pd
import pickle
import networkx as nx
import numpy as np
import scipy as sp
import tqdm

In [6]:
pd.options.display.max_columns=100

In [3]:
with open('./data/original_tweets.p', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.head()

,contributors,coordinates,created_at,display_text_range,favorite_count,favorited,filter_level,geo,hashtags,id,...,retweet_count,retweeted,scopes,source,text,timestamp_ms,truncated,user,username,withheld_in_countries
0,None,None,2018-10-28 23:35:48+00:00,"[0, 109]",0,False,low,None,"[{'text': 'Warszawa', 'indices': [76, 85]}]",1056691029846491139,...,0,False,NaN,"<a href=""https://echowarszawy.pl"" rel=""nofollo...",KMPSP Warszawa: 23.10.2018 - Drzewo powalone n...,1540769748542,False,965943739809239040,EchoWarszawy,NaN
1,None,None,2018-10-28 23:35:51+00:00,"[48, 290]",0,False,low,None,[],1056691042106400768,...,0,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@SebNow82 @ChiefRobert2 @Immanuela_Kant @smok0...,1540769751465,True,870751323549040640,dodi6319,NaN
2,None,None,2018-10-28 23:35:56+00:00,"[50, 330]",0,False,low,None,[],1056691065200304128,...,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@rafalch2 @KLubnauer @Nowoczesna @wlodekczarza...,1540769756971,True,1025517195223281664,Fiona11701355,NaN
3,None,None,2018-10-28 23:35:59+00:00,"[20, 72]",0,False,low,None,[],1056691075727982592,...,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@wkrawcz1 @pisorgpl W PiS to normalne. Wszyscy...,1540769759481,False,811556244892872704,Paul4Anka,NaN
4,None,None,2018-10-28 23:36:10+00:00,NaN,0,False,low,None,[],1056691121097752576,...,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","Czasami żałuję, że Krysia mnie zablokowała😘",1540769770298,False,1004022955045728256,JanKiebasa3,NaN


In [14]:
df['hashtags'] = df['hashtags'].apply(lambda x: [item['text'] for item in x])

In [16]:
df['embedding'] = df.apply(lambda row: np.random.randn(100), axis=1)

In [38]:
G = nx.Graph()

for row in tqdm.tqdm_notebook(df.itertuples(), total=len(df)):
    if row.hashtags:
        G.add_node(row.id)
        G.node[row.id]['embedding'] = row.embedding
        G.node[row.id]['node_type'] = 'tweet'
        for hashtag in row.hashtags:
            G.add_node(hashtag)
            G.node[hashtag]['node_type'] = 'hashtag'
            G.add_edge(row.id, hashtag)

In [39]:
G.order(), G.size()

(69131, 117903)

In [49]:
for node in tqdm.tqdm_notebook(G.nodes):
    if G.nodes[node]['node_type'] == 'hashtag':
        tweets = G.neighbors(node)
        embeddings = np.asarray([G.node[tweet]['embedding'] for tweet in tweets])
        G.node[node]['embedding'] = embeddings.mean(axis=0)

In [50]:
for node in G.nodes:
    assert 'embedding' in G.node[node]

In [66]:
min_dist = np.inf
max_dist = - np.inf

min_sim = np.inf
max_sim = - np.inf

for node_from, node_to, edge_features in G.edges(data=True):
    emb_from = G.node[node_from]['embedding']
    emb_to = G.node[node_to]['embedding']
    
    distance = sp.spatial.distance.cosine(emb_from, emb_to)
    similarity = 1 - distance
    ang_dist = np.arccos(similarity) / np.pi
    ang_sim = 1 - ang_dist
    
    edge_features['distance'] = ang_dist
    edge_features['similarity'] = ang_sim
    
    max_dist = max(max_dist, edge_features['distance'])
    min_dist = min(min_dist, edge_features['distance'])
    max_sim = max(max_sim, ang_sim)
    min_sim = min(min_sim, ang_sim)

In [67]:
min_dist, max_dist

(0.0, 0.6416105063927715)

In [68]:
min_sim, max_sim

(0.3583894936072285, 1.0)

In [69]:
x = np.random.randn(100, 5)

In [80]:
(x * x).sum(1).shape

(100,)

In [74]:
x.dot(x.T)

array([[ 5.48744863, -3.42423236,  2.82947632, ..., -1.93913727,
         0.39894769, -2.18255174],
       [-3.42423236,  8.88845169, -4.19005036, ...,  9.30107261,
        -3.00564674, -1.7772976 ],
       [ 2.82947632, -4.19005036,  4.38883791, ..., -4.26971332,
         1.66691357,  0.65354866],
       ...,
       [-1.93913727,  9.30107261, -4.26971332, ..., 13.92336166,
        -4.82415801, -5.81693337],
       [ 0.39894769, -3.00564674,  1.66691357, ..., -4.82415801,
         3.31029953,  1.47211522],
       [-2.18255174, -1.7772976 ,  0.65354866, ..., -5.81693337,
         1.47211522,  5.06702715]])